# Instalacion de paquetes necesarios

In [ ]:
!pip install langchain chromadb pypdf

# Caso de uso

Vamos a cargar un pdf con el manual del juego magin al cual le realizaremos algunas preguntas

## importacion de paquetes necesarios

In [1]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

## Cargamos el documento pdf

In [3]:
loader = PyPDFLoader(".\Doc\mtg.pdf")
pages = loader.load_and_split()

In [4]:
# Un elemento por cada página
pages[3].page_content

'Jasper SandnerC 214/ 269 2/1Oo2\n™ & © 2014 Wizards of the CoastSP• M15Campo de \nbatallatú \n16 vidas\nrestantes\nyo\n18 vidas \nrestantesCementerio  Biblioteca  \nBiblioteca  Mano\nManoCementerio  \n3Para comenzar el juego, baraja tu mazo, \ntambién conocido como tu biblioteca. Roba una mano de siete cartas y comprueba cuántas tierras tienes. Puedes mirar la línea de texto que hay bajo la ilustración de cada carta para ver de qué tipo de carta se trata. Para este primer juego, si no tienes al menos dos tierras, baraja de nuevo tu mazo (incluyendo tu mano anterior) y roba una mano nueva.\nCada jugador comienza con 20 vidas, \ny cada uno debe llevar la cuenta de su total de vidas de alguna manera (con un dado, lápiz y papel...). ¡Reduce el total de vidas de tu oponente a 0 y ganarás el juego!Comenzar'

In [5]:
# Objeto que va a hacer los cortes en el texto
split = CharacterTextSplitter(chunk_size=240, separator = '.\n')

In [6]:
textos = split.split_documents(pages) # Lista de textos

Created a chunk of size 288, which is longer than the specified 240
Created a chunk of size 364, which is longer than the specified 240
Created a chunk of size 326, which is longer than the specified 240
Created a chunk of size 325, which is longer than the specified 240
Created a chunk of size 503, which is longer than the specified 240
Created a chunk of size 1507, which is longer than the specified 240
Created a chunk of size 308, which is longer than the specified 240
Created a chunk of size 583, which is longer than the specified 240
Created a chunk of size 458, which is longer than the specified 240
Created a chunk of size 429, which is longer than the specified 240
Created a chunk of size 626, which is longer than the specified 240
Created a chunk of size 358, which is longer than the specified 240
Created a chunk of size 892, which is longer than the specified 240
Created a chunk of size 1311, which is longer than the specified 240
Created a chunk of size 333, which is longer t

In [7]:
textos[0].page_content # Texto de la primera página
textos[0] # Objeto de tipo Texto

Document(page_content='Guía de inicio \nrápidoEdad: 13 o +', metadata={'source': '.\\Doc\\mtg.pdf', 'page': 0})

## Carga motor de embedding

In [2]:
embedding_function = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
# Equivalent to SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

## Base de datos

Esta base de datos cuenta con persistencia de datos locales en el directorio `./datamtg`

### Guardar en el local

In [148]:
db = Chroma.from_documents(textos, embedding_function,  persist_directory="./datamtg")

### Cargar base de datos

In [3]:
db = Chroma(persist_directory="./datamtg", embedding_function=embedding_function)

## consultas

### Consulta simple

In [5]:
query = "Con cuanta vida empiezan los jugadores?"
docs = db.similarity_search(query)
docs

[Document(page_content='Cada jugador comienza con 20 vidas, \ny cada uno debe llevar la cuenta de su total de vidas de alguna manera (con un dado, lápiz y papel...). ¡Reduce el total de vidas de tu oponente a 0 y ganarás el juego!Comenzar', metadata={'page': 2, 'source': '.\\Doc\\mtg.pdf'}),
 Document(page_content='Cada jugador comienza con 20 vidas, \ny cada uno debe llevar la cuenta de su total de vidas de alguna manera (con un dado, lápiz y papel...). ¡Reduce el total de vidas de tu oponente a 0 y ganarás el juego!Comenzar', metadata={'page': 2, 'source': '.\\Doc\\mtg.pdf'}),
 Document(page_content='∙Paso de final del combate.\nFase principal (de nuevo)\n Puedes jugar una tierra si aún no has jugado ninguna.  \nPuedes lanzar hechizos', metadata={'page': 15, 'source': '.\\Doc\\mtg.pdf'}),
 Document(page_content='∙Paso de final del combate.\nFase principal (de nuevo)\n Puedes jugar una tierra si aún no has jugado ninguna.  \nPuedes lanzar hechizos', metadata={'page': 15, 'source': '.\

### Comsulta con puntajes

El puntaje mas pequeño es el mejor

In [6]:
query = "Con cuanta vida empiezan los jugadores?"
docs = db.similarity_search_with_score(query)
docs

[(Document(page_content='Cada jugador comienza con 20 vidas, \ny cada uno debe llevar la cuenta de su total de vidas de alguna manera (con un dado, lápiz y papel...). ¡Reduce el total de vidas de tu oponente a 0 y ganarás el juego!Comenzar', metadata={'page': 2, 'source': '.\\Doc\\mtg.pdf'}),
  0.6259155869483948),
 (Document(page_content='Cada jugador comienza con 20 vidas, \ny cada uno debe llevar la cuenta de su total de vidas de alguna manera (con un dado, lápiz y papel...). ¡Reduce el total de vidas de tu oponente a 0 y ganarás el juego!Comenzar', metadata={'page': 2, 'source': '.\\Doc\\mtg.pdf'}),
  0.6259155869483948),
 (Document(page_content='∙Paso de final del combate.\nFase principal (de nuevo)\n Puedes jugar una tierra si aún no has jugado ninguna.  \nPuedes lanzar hechizos', metadata={'page': 15, 'source': '.\\Doc\\mtg.pdf'}),
  0.7695561647415161),
 (Document(page_content='∙Paso de final del combate.\nFase principal (de nuevo)\n Puedes jugar una tierra si aún no has jugado

# Operaciones

## Listar colecciones

In [ ]:
colecciones = db._collection.get()

colecciones

In [ ]:
colecciones = db.get()

colecciones

## Filtros

Al igual que en su cliente chromaDB permite usar los siguientes filtros

* ids: El id del documento a traer. Optional.
* where: diccionario con los filtros a utilizar Ej. {"$and": ["color" : "red", "price": {"$gte": 4.20}]}. Optional.
* limit: El numero limite de documentos a devulver. Optional.
* offset: El desplazamiento a partir del cual se devolverán los resultados. Útil para paginar resultados con límite. Optional.
* where_document: Un diccionario de tipo WhereDocument sirve para filtrar por los documentos. Ej. {$contains: {"text": "hello"}}. Optional.
* include: Una lista de lo que debe incluirse en los resultados, los identificadores se agregas por default Ej. ["metadatas", "documents"]. Optional

### Filtrar por metadata

para filtrar por metadata utilizaremos where y estas son las operaciones permitidas

* $eq - igual a (cadena, int, flotante)
* $ne - no igual a (cadena, int, flotante)
* $gt - mayor que (int, float)
* $gte - mayor o igual que (int, float)
* $lt - menor que (int, float)
* $lte - menor o igual que (int, float)

In [96]:
# Filtrar todas las colecciones que no sean la de la página 3
db.get(where={'page': 3 })

{'ids': ['5c199552-9a8c-11ee-b2eb-d45d64279660',
  '5c199553-9a8c-11ee-bd63-d45d64279660',
  '5c199554-9a8c-11ee-8c0b-d45d64279660',
  '5c199555-9a8c-11ee-ba11-d45d64279660',
  'a6357b9e-9a8e-11ee-bce5-d45d64279660',
  'a6357b9f-9a8e-11ee-a9d8-d45d64279660',
  'a6357ba0-9a8e-11ee-9334-d45d64279660',
  'a6357ba1-9a8e-11ee-beee-d45d64279660'],
 'embeddings': None,
 'metadatas': [{'page': 3, 'source': '.\\Doc\\mtg.pdf'},
  {'page': 3, 'source': '.\\Doc\\mtg.pdf'},
  {'page': 3, 'source': '.\\Doc\\mtg.pdf'},
  {'page': 3, 'source': '.\\Doc\\mtg.pdf'},
  {'page': 3, 'source': '.\\Doc\\mtg.pdf'},
  {'page': 3, 'source': '.\\Doc\\mtg.pdf'},
  {'page': 3, 'source': '.\\Doc\\mtg.pdf'},
  {'page': 3, 'source': '.\\Doc\\mtg.pdf'}],
 'documents': ['Tierra básica — BosqueBosque\nJonas De RoL269/ 269\nSP• M15™ & © 2014 Wizards of the Coast\nLlanura oW (blanco)\nIsla oU (azul)\nPantano oB (negro)\nMontaña oR (rojo)\nBosque oG (verde)Maná que produce\n Tipo de tierra básica\n4Tu mazo incluye tierras y

### Filtros por documento

Podemos filtrar por el contenido de un documento utilizando ``where_document``

In [147]:
# Filtrar todos los documentos que contengan la palabra "Para comenzar el juego"

db.get(where_document={'$contains': 'Para comenzar el juego' })

{'ids': ['2e72ab19-9a9e-11ee-8f1d-d45d64279660'],
 'embeddings': None,
 'metadatas': [{'page': 2, 'source': '.\\Doc\\mtg.pdf'}],
 'documents': ['Jasper SandnerC 214/ 269 2/1Oo2\n™ & © 2014 Wizards of the CoastSP• M15Campo de \nbatallatú \n16 vidas\nrestantes\nyo\n18 vidas \nrestantesCementerio  Biblioteca  \nBiblioteca  Mano\nManoCementerio  \n3Para comenzar el juego, baraja tu mazo, \ntambién conocido como tu biblioteca. Roba una mano de siete cartas y comprueba cuántas tierras tienes. Puedes mirar la línea de texto que hay bajo la ilustración de cada carta para ver de qué tipo de carta se trata. Para este primer juego, si no tienes al menos dos tierras, baraja de nuevo tu mazo (incluyendo tu mano anterior) y roba una mano nueva'],
 'uris': None,
 'data': None}

### Usar tanto where como where_document

In [124]:
#Creamos la consulta

# Consulta para filtrar todos los documentos que no sean mayores o iguales a 2 , menores o iguales a 5 y que contengan la palabra "hechizos"

db.get(where={'$and' : [
    { 'page' : {'$gte': 2}},
    {'page' : {'$lte': 5 }}
]}, where_document={'$contains': 'hechizos' })

{'ids': ['5c19955d-9a8c-11ee-9809-d45d64279660',
  '5c19955e-9a8c-11ee-a402-d45d64279660',
  '5c19955f-9a8c-11ee-9908-d45d64279660',
  '5c199563-9a8c-11ee-9f9e-d45d64279660',
  'a6357ba9-9a8e-11ee-9df2-d45d64279660',
  'a6357baa-9a8e-11ee-b548-d45d64279660',
  'a6357bab-9a8e-11ee-951e-d45d64279660',
  'a6357baf-9a8e-11ee-98f3-d45d64279660'],
 'embeddings': None,
 'metadatas': [{'page': 4, 'source': '.\\Doc\\mtg.pdf'},
  {'page': 4, 'source': '.\\Doc\\mtg.pdf'},
  {'page': 4, 'source': '.\\Doc\\mtg.pdf'},
  {'page': 5, 'source': '.\\Doc\\mtg.pdf'},
  {'page': 4, 'source': '.\\Doc\\mtg.pdf'},
  {'page': 4, 'source': '.\\Doc\\mtg.pdf'},
  {'page': 4, 'source': '.\\Doc\\mtg.pdf'},
  {'page': 5, 'source': '.\\Doc\\mtg.pdf'}],
 'documents': ['“Y si miento… ”, comenzó a decir.ArtefactoMeteorito\nScott MurphyU\n221/ 269Oo5\nSP•M15™ & © 2014 Wizards of the Coast\n5Cualquier carta que no sea una tierra puede lanzarse como un hechizo. Algunos \ntipos de hechizos se ponen en el campo de batalla y 

## Actualizacion de documento

Vamos actualizar los metadatos del primer indice del resultado

In [80]:
# Obtenemos el texto de la página
text = docs[0].page_content

In [88]:
# Obtenemos su id correspondiente en la base de datos
idtoUpdate = db.get(where_document={'$contains': '{}'.format(text) })['ids'][0]

In [89]:
# Del resultado modificamos los metadatos

docs[0].metadata = {'page': 2, 'source': '.\\Doc\\mtg.pdf', 'category': 'mtg_rules'}

In [90]:
# Actualiza el documento en ChromaDB
db.update_document(idtoUpdate, docs[0])

In [92]:
# Verificamos que se haya actualizado
db.get(where_document={'$contains': '{}'.format(text) })

{'ids': ['5c199551-9a8c-11ee-b6f7-d45d64279660',
  'a6357b9d-9a8e-11ee-8d58-d45d64279660'],
 'embeddings': None,
 'metadatas': [{'category': 'mtg_rules',
   'page': 2,
   'source': '.\\Doc\\mtg.pdf'},
  {'page': 2, 'source': '.\\Doc\\mtg.pdf'}],
 'documents': ['Cada jugador comienza con 20 vidas, \ny cada uno debe llevar la cuenta de su total de vidas de alguna manera (con un dado, lápiz y papel...). ¡Reduce el total de vidas de tu oponente a 0 y ganarás el juego!Comenzar',
  'Cada jugador comienza con 20 vidas, \ny cada uno debe llevar la cuenta de su total de vidas de alguna manera (con un dado, lápiz y papel...). ¡Reduce el total de vidas de tu oponente a 0 y ganarás el juego!Comenzar'],
 'uris': None,
 'data': None}

## Eliminar documentos

Vamos a eliminar un documento, esta accion se puede hacer tanto con los ids como los demas datos del documento y incluso una consulta

En este caso vamos a eliminar todos los documentos que cumplan con nuestra clusula where

In [125]:
print("antes de la eliminacion", db._collection.count())

antes de la eliminacion 236


In [126]:
db._collection.delete(where={'$and' : [
    { 'page' : {'$gte': 2}},
    {'page' : {'$lte': 5 }}
]}, where_document={'$contains': 'hechizos' })

In [127]:
print("despues de la eliminacion", db._collection.count())

despues de la eliminacion 228


In [128]:
# Confirma que se haya eliminado

db.get(where={'$and' : [
    { 'page' : {'$gte': 2}},
    {'page' : {'$lte': 5 }}
]}, where_document={'$contains': 'hechizos' })

{'ids': [],
 'embeddings': None,
 'metadatas': [],
 'documents': [],
 'uris': None,
 'data': None}

## Eliminar la coleccion

In [140]:
db.delete_collection()